In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.8.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\ryanr\appdata\local\programs\python\python310\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, protobuf, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, tf-estimator-nightly, typing-extensions, wrapt
Required-by: tensorflow-text


In [10]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [33]:
MAX_SEQ_LENGTH = 30
NUM_FEATURES = 258

EPOCHS = 100

DATA_PATH = "transformer_data_noface_seq30_originalactions"

sequence_length = 30

# actions = ['can','you','help','me','what','name','hamburger','french fries','thanks','bye','hello','excuse me','sorry'] original


In [20]:
MAX_SEQ_LENGTH = 60
NUM_FEATURES = 258

EPOCHS = 500

DATA_PATH = "transformer_data_noface_seq60_16actions"

sequence_length = 60

actions = ['can','you','help','me','your','name','what','hamburger','french fries','hello','bye','excuse me','sorry','water','thanks','yesterday']

In [14]:
train_data, train_labels = np.load(os.path.join(DATA_PATH,"train_data.npy")), np.load(os.path.join(DATA_PATH,"train_labels.npy"))
test_data, test_labels = np.load(os.path.join(DATA_PATH,"test_data.npy")), np.load(os.path.join(DATA_PATH,"test_labels.npy"))

print(f"Frame features in train set: {train_data.shape}")

Frame features in train set: (1382, 60, 258)


In [15]:
assert not np.any(np.isnan(train_data))
assert not np.any(np.isnan(train_labels))
assert not np.any(np.isnan(test_data))
assert not np.any(np.isnan(test_labels))

## Build Transformer

In [21]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask
    def get_config(self):
        config = super().get_config()
        config.update({
#             "position_embeddings": self.position_embeddings,
            "sequence_length": self.sequence_length,
            "output_dim": self.output_dim,
        })
        return config

In [22]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
#             "attention": self.attention,
#             "dense_proj": self.dense_proj,
#             "layernorm_1": self.layernorm_1,
#             "layernorm_2": self.layernorm_2,
        })
        return config

In [23]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 8
    num_heads = 6
    classes = len(actions)

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment(filepath):
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        validation_split=0.15,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

def load_model(filepath):
    model = get_compiled_model()
    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model


## Load or Train Model

In [24]:
trained_model = run_experiment("transformer_model_withface_seq60_16actions") # arg: save path

Epoch 1/500
34/37 [==========================>...] - ETA: 0s - loss: 3.5043 - accuracy: 0.0928
Epoch 1: val_loss improved from inf to 2.65615, saving model to transformer_model_withface_seq60_16actions
37/37 [==============================] - 3s 51ms/step - loss: 3.4667 - accuracy: 0.0971 - val_loss: 2.6562 - val_accuracy: 0.0913
Epoch 2/500
34/37 [==========================>...] - ETA: 0s - loss: 2.8678 - accuracy: 0.1562
Epoch 2: val_loss improved from 2.65615 to 2.03147, saving model to transformer_model_withface_seq60_16actions
37/37 [==============================] - 2s 48ms/step - loss: 2.8486 - accuracy: 0.1593 - val_loss: 2.0315 - val_accuracy: 0.3750
Epoch 3/500
35/37 [===========================>..] - ETA: 0s - loss: 2.1441 - accuracy: 0.3009
Epoch 3: val_loss improved from 2.03147 to 1.47480, saving model to transformer_model_withface_seq60_16actions
37/37 [==============================] - 1s 34ms/step - loss: 2.1185 - accuracy: 0.3101 - val_loss: 1.4748 - val_accuracy: 0.5

34/37 [==========================>...] - ETA: 0s - loss: 0.1705 - accuracy: 0.9412
Epoch 28: val_loss did not improve from 0.43078
37/37 [==============================] - 1s 15ms/step - loss: 0.1662 - accuracy: 0.9412 - val_loss: 0.5260 - val_accuracy: 0.8413
Epoch 29/500
34/37 [==========================>...] - ETA: 0s - loss: 0.1382 - accuracy: 0.9522
Epoch 29: val_loss did not improve from 0.43078
37/37 [==============================] - 1s 16ms/step - loss: 0.1406 - accuracy: 0.9497 - val_loss: 0.6141 - val_accuracy: 0.8221
Epoch 30/500
37/37 [==============================] - ETA: 0s - loss: 0.1264 - accuracy: 0.9574
Epoch 30: val_loss did not improve from 0.43078
37/37 [==============================] - 1s 16ms/step - loss: 0.1264 - accuracy: 0.9574 - val_loss: 0.4482 - val_accuracy: 0.8750
Epoch 31/500
34/37 [==========================>...] - ETA: 0s - loss: 0.1352 - accuracy: 0.9513
Epoch 31: val_loss did not improve from 0.43078
37/37 [==============================] - 1s 15m

37/37 [==============================] - 1s 14ms/step - loss: 0.1822 - accuracy: 0.9421 - val_loss: 0.7607 - val_accuracy: 0.8125
Epoch 58/500
33/37 [=========================>....] - ETA: 0s - loss: 0.1434 - accuracy: 0.9479
Epoch 58: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 14ms/step - loss: 0.1400 - accuracy: 0.9480 - val_loss: 0.6653 - val_accuracy: 0.8413
Epoch 59/500
34/37 [==========================>...] - ETA: 0s - loss: 0.0746 - accuracy: 0.9752
Epoch 59: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 14ms/step - loss: 0.0729 - accuracy: 0.9761 - val_loss: 0.5086 - val_accuracy: 0.8750
Epoch 60/500
34/37 [==========================>...] - ETA: 0s - loss: 0.0644 - accuracy: 0.9770
Epoch 60: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 14ms/step - loss: 0.0627 - accuracy: 0.9770 - val_loss: 0.5473 - val_accuracy: 0.8558
Epoch 61/500
33/37 [=========================>..

37/37 [==============================] - 1s 15ms/step - loss: 0.0650 - accuracy: 0.9796 - val_loss: 0.5835 - val_accuracy: 0.8510
Epoch 88/500
36/37 [============================>.] - ETA: 0s - loss: 0.0445 - accuracy: 0.9844
Epoch 88: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 15ms/step - loss: 0.0438 - accuracy: 0.9847 - val_loss: 0.4826 - val_accuracy: 0.8654
Epoch 89/500
35/37 [===========================>..] - ETA: 0s - loss: 0.0299 - accuracy: 0.9929
Epoch 89: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 15ms/step - loss: 0.0292 - accuracy: 0.9932 - val_loss: 0.5516 - val_accuracy: 0.8269
Epoch 90/500
36/37 [============================>.] - ETA: 0s - loss: 0.0508 - accuracy: 0.9835
Epoch 90: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 15ms/step - loss: 0.0505 - accuracy: 0.9838 - val_loss: 0.6651 - val_accuracy: 0.8317
Epoch 91/500
36/37 [============================

33/37 [=========================>....] - ETA: 0s - loss: 0.0251 - accuracy: 0.9915
Epoch 117: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 16ms/step - loss: 0.0277 - accuracy: 0.9915 - val_loss: 0.5759 - val_accuracy: 0.8654
Epoch 118/500
35/37 [===========================>..] - ETA: 0s - loss: 0.0140 - accuracy: 0.9964
Epoch 118: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 15ms/step - loss: 0.0139 - accuracy: 0.9966 - val_loss: 0.7175 - val_accuracy: 0.8606
Epoch 119/500
33/37 [=========================>....] - ETA: 0s - loss: 0.0164 - accuracy: 0.9953
Epoch 119: val_loss did not improve from 0.36728
37/37 [==============================] - 1s 15ms/step - loss: 0.0165 - accuracy: 0.9957 - val_loss: 0.6323 - val_accuracy: 0.8413
Epoch 120/500
35/37 [===========================>..] - ETA: 0s - loss: 0.0242 - accuracy: 0.9920
Epoch 120: val_loss did not improve from 0.36728
37/37 [==============================] -

...

Epoch 356/500
34/37 [==========================>...] - ETA: 0s - loss: 0.0071 - accuracy: 0.9991
Epoch 356: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 14ms/step - loss: 0.0066 - accuracy: 0.9991 - val_loss: 0.4378 - val_accuracy: 0.8990
Epoch 357/500
35/37 [===========================>..] - ETA: 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 357: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 15ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.4141 - val_accuracy: 0.9135
Epoch 358/500
36/37 [============================>.] - ETA: 0s - loss: 9.1700e-04 - accuracy: 1.0000
Epoch 358: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 14ms/step - loss: 9.3352e-04 - accuracy: 1.0000 - val_loss: 0.4856 - val_accuracy: 0.9135
Epoch 359/500
36/37 [============================>.] - ETA: 0s - loss: 0.0017 - accuracy: 0.9991
Epoch 359: val_loss did not improve from 0.33951
37/37 [===========

37/37 [==============================] - 1s 15ms/step - loss: 0.0379 - accuracy: 0.9864 - val_loss: 1.0814 - val_accuracy: 0.7933
Epoch 386/500
33/37 [=========================>....] - ETA: 0s - loss: 0.0421 - accuracy: 0.9858
Epoch 386: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 14ms/step - loss: 0.0485 - accuracy: 0.9838 - val_loss: 0.6961 - val_accuracy: 0.8558
Epoch 387/500
33/37 [=========================>....] - ETA: 0s - loss: 0.0253 - accuracy: 0.9905
Epoch 387: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 14ms/step - loss: 0.0320 - accuracy: 0.9872 - val_loss: 0.6538 - val_accuracy: 0.8558
Epoch 388/500
34/37 [==========================>...] - ETA: 0s - loss: 0.0234 - accuracy: 0.9926
Epoch 388: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 14ms/step - loss: 0.0224 - accuracy: 0.9932 - val_loss: 0.5894 - val_accuracy: 0.8558
Epoch 389/500
35/37 [=====================

35/37 [===========================>..] - ETA: 0s - loss: 5.9874e-04 - accuracy: 1.0000
Epoch 415: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 15ms/step - loss: 7.9812e-04 - accuracy: 1.0000 - val_loss: 0.4658 - val_accuracy: 0.9183
Epoch 416/500
37/37 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000    
Epoch 416: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 14ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.5114 - val_accuracy: 0.9135
Epoch 417/500
34/37 [==========================>...] - ETA: 0s - loss: 0.0056 - accuracy: 0.9982
Epoch 417: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 15ms/step - loss: 0.0053 - accuracy: 0.9983 - val_loss: 0.5635 - val_accuracy: 0.8990
Epoch 418/500
35/37 [===========================>..] - ETA: 0s - loss: 0.0092 - accuracy: 0.9982
Epoch 418: val_loss did not improve from 0.33951
37/37 [=====================

Epoch 445/500
37/37 [==============================] - ETA: 0s - loss: 0.0569 - accuracy: 0.9838
Epoch 445: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 16ms/step - loss: 0.0569 - accuracy: 0.9838 - val_loss: 0.7804 - val_accuracy: 0.8413
Epoch 446/500
36/37 [============================>.] - ETA: 0s - loss: 0.0261 - accuracy: 0.9887
Epoch 446: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 21ms/step - loss: 0.0263 - accuracy: 0.9889 - val_loss: 0.6863 - val_accuracy: 0.8221
Epoch 447/500
34/37 [==========================>...] - ETA: 0s - loss: 0.0287 - accuracy: 0.9899
Epoch 447: val_loss did not improve from 0.33951
37/37 [==============================] - 2s 64ms/step - loss: 0.0284 - accuracy: 0.9898 - val_loss: 0.6227 - val_accuracy: 0.8221
Epoch 448/500
37/37 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 0.9957
Epoch 448: val_loss did not improve from 0.33951
37/37 [===================

Epoch 475/500
37/37 [==============================] - ETA: 0s - loss: 0.0259 - accuracy: 0.9889
Epoch 475: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 15ms/step - loss: 0.0259 - accuracy: 0.9889 - val_loss: 0.4372 - val_accuracy: 0.8990
Epoch 476/500
36/37 [============================>.] - ETA: 0s - loss: 0.0099 - accuracy: 0.9983
Epoch 476: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 15ms/step - loss: 0.0098 - accuracy: 0.9983 - val_loss: 0.6850 - val_accuracy: 0.8413
Epoch 477/500
36/37 [============================>.] - ETA: 0s - loss: 0.0081 - accuracy: 0.9983
Epoch 477: val_loss did not improve from 0.33951
37/37 [==============================] - 1s 16ms/step - loss: 0.0080 - accuracy: 0.9983 - val_loss: 0.5626 - val_accuracy: 0.8654
Epoch 478/500
35/37 [===========================>..] - ETA: 0s - loss: 0.0073 - accuracy: 0.9982
Epoch 478: val_loss did not improve from 0.33951
37/37 [===================

In [25]:
trained_model = load_model("transformer_model_withface_seq60_16actions") # arg: load path

5/5 [==============================] - 1s 32ms/step - loss: 0.1548 - accuracy: 0.9610
Test accuracy: 96.1%


In [59]:
del trained_model

In [55]:
trained_model.save("test.h5")

In [58]:
trained_model.to_json()

'{"class_name": "Functional", "config": {"name": "model_7", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, null, null], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_8"}, "name": "input_8", "inbound_nodes": []}, {"class_name": "PositionalEmbedding", "config": {"name": "frame_position_embedding", "trainable": true, "dtype": "float32", "sequence_length": 30, "output_dim": 258}, "name": "frame_position_embedding", "inbound_nodes": [[["input_8", 0, 0, {}]]]}, {"class_name": "TransformerEncoder", "config": {"name": "transformer_layer", "trainable": true, "dtype": "float32", "embed_dim": 258, "dense_dim": 8, "num_heads": 6}, "name": "transformer_layer", "inbound_nodes": [[["frame_position_embedding", 0, 0, {}]]]}, {"class_name": "GlobalMaxPooling1D", "config": {"name": "global_max_pooling1d_7", "trainable": true, "dtype": "float32", "data_format": "channels_last", "keepdims": false}, "name": "global_max_pooling1d_7", "inbound_nodes": [[[

In [56]:
tf.saved_model.save(trained_model,'saved_model/transformer')

INFO:tensorflow:Assets written to: saved_model/transformer\assets


INFO:tensorflow:Assets written to: saved_model/transformer\assets


In [9]:
!python -m tf2onnx.convert --saved-model transformer_model --output model.onnx

C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-01-19 23:17:58,232 - WARNING - '--tag' not specified for saved_model. Using --tag serve
Traceback (most recent call last):
  File "C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\site-packages\tf2onnx\convert.py", line 696, in <module>
    main()
  File "C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\site-packages\tf2onnx\convert.py", line 235, in main
    graph_def, inputs, outputs, initialized_tables, 

## Run it real time

In [26]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [27]:
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils # drawing utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [28]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [29]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            # Read feed
            ret, image = cap.read()

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_pose = pose.process(image)


            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



    #             print(len(results.multi_hand_landmarks))
            hand_np = []
            if results.multi_hand_landmarks:
                for i in range(len(results.multi_hand_landmarks)):
#                     print(i)
                    mp_drawing.draw_landmarks(image,results.multi_hand_landmarks[i],mp_hands.HAND_CONNECTIONS,mp_drawing_styles.get_default_hand_landmarks_style(),mp_drawing_styles.get_default_hand_connections_style())
                    hand_np.append(np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[i].landmark]).flatten() if results.multi_hand_landmarks[i] else np.zeros(21*3))

            if len(hand_np) == 0:
                lh = np.zeros(21*3)
                rh = np.zeros(21*3)
            elif len(hand_np) == 1:
                lh = hand_np[0]
                rh = np.zeros(21*3)
            else:
                lh = hand_np[0]
                rh = hand_np[1]        

            # Draw pose annotations        
            mp_drawing.draw_landmarks(image,results_pose.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            ps = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)

            keypoints = np.concatenate([ps, lh, rh])
            sequence.append(keypoints)
            sequence = sequence[-sequence_length:]

            if len(sequence) == sequence_length:
                res = trained_model.predict(np.expand_dims(sequence, axis=0))[0]
#                 print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))


                #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 4: 
                    sentence = sentence[-4:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)
                
#                 sequence = []
#                 cv2.waitKey(1000)
                

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()